In [2]:
import sys
import json
import numpy as np
from tqdm.notebook import tqdm
from toolz.curried import pipe, curry, compose

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader

import chnet.ch_tools as ch_tools
import chnet.utilities as ch_utils
import chnet.ch_generator as ch_gen
from chnet.ch_loader import CahnHillDataset
from chnet.models import UNet, UNet_solo_loop, UNet_loop, mse_loss

In [55]:
def train(key="unet", mid=0.0, dif=0.449, dim_x=96, dx=0.25, dt=0.01, 
            gamma=0.2, init_steps=1, nstep=20, n_samples_trn=1024, 
            ngf=32, final_tstep = 5000, num_epochs=10, 
            learning_rate=1.0e-5, n_primes=2000, 
            device="cuda"):
    
    m_l=mid-dif
    m_r=mid+dif
    delta_sim_steps=(final_tstep-init_steps)//nstep
    primes = ch_utils.get_primes(n_primes)
    
    print("no. of datasets: {}".format(len(primes)))
    
    device = torch.device("cuda:0") if device == "cuda" else torch.device("cpu")
    print(device)
    if key == "unet":
        model=UNet(in_channels=1, out_channels=1, init_features=ngf, tanh=False).double().to(device)
    elif key == "unet_solo_loop":
        model=UNet_solo_loop(in_channels=1, out_channels=1, init_features=ngf, temporal=nstep, tanh=False).double().to(device)
    elif key == "unet_loop":
        model=UNet_loop(in_channels=1, out_channels=1, init_features=ngf, temporal=nstep, tanh=False).double().to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    trn_losses = []

    fout = "weights/model_{}_size_{}_step_{}_init_{}_delta_{}_tstep_{}.pt".format(key, ngf, nstep, init_steps, delta_sim_steps, num_epochs*len(primes))  
    print("model saved at: {}".format(fout))

    print("Start Training")
    for num, prime in enumerate(primes):
        # Loss and optimizer
        torch.cuda.empty_cache()
        x_trn, y_trn = ch_gen.data_generator(nsamples=n_samples_trn, 
                                      dim_x=dim_x, 
                                      init_steps=init_steps, 
                                      delta_sim_steps = delta_sim_steps,
                                      dx=dx, 
                                      dt=dt,
                                      m_l=m_l, 
                                      m_r=m_r,
                                      n_step=nstep,
                                      gamma=gamma, 
                                      seed=2513*prime,
                                      device=device)


        trn_dataset = CahnHillDataset(x_trn, y_trn, 
                                      transform_x=lambda x: x[:,None], 
                                      transform_y=lambda x: x[:,None])

        trn_loader = DataLoader(trn_dataset, 
                                batch_size=8, 
                                shuffle=True, 
                                num_workers=4)

        print("Training Run: {}, prime: {}".format(num, prime))

        total_step = len(trn_loader)
        
        for epoch in range(num_epochs):  
            for i, item_trn in enumerate(tqdm(trn_loader)):
                
                model.train()
                
                if "loop" in key:
                    if "solo" in key:
                        x = item_trn['x'][:,0].to(device)
                    else:
                        x = item_trn['x'][:,0].to(device)
                    y_tru = item_trn['y'].to(device)
                else:
                    x = item_trn['x'][:,0].to(device)
                    y_tru = item_trn['y'][:,-1] .to(device) 
                
                y_prd = model(x)# Forward pass
                means_inp = x.mean(axis=(1,2,3))
                means_out = y_prd.mean(axis=(1,2,3))
                loss = mse_loss(y_tru, y_prd, scale=10000) + mse_loss(means_inp, means_out, scale=10000)
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                trn_losses.append(np.sqrt(loss.item()))

            print ('Epoch [{}/{}], Training Loss: {:.11f}'.format(epoch+1, num_epochs, np.mean(trn_losses[-total_step:])))

#         obj = {}
#         obj["state"] = model.state_dict()
#         obj["losses"] = trn_losses
#         torch.save(obj, fout)
    print("End Training")
    return model

In [56]:
arguments = {"key":"unet",
             "mid":0.0, 
             "dif":0.449, 
             "dim_x":96,
             "init_steps":1, 
             "dx":0.25,
             "dt":0.01,
             "gamma":0.2, 
             "n_samples_trn":1024,
             "ngf":64,
             "nstep":2,
             "final_tstep":501,
             "num_epochs":10,
             "n_primes":100,
             "learning_rate":1.0e-5,
             "device":"cuda"}

In [57]:
print(arguments)
model = train(**arguments)

{'key': 'unet', 'mid': 0.0, 'dif': 0.449, 'dim_x': 96, 'init_steps': 1, 'dx': 0.25, 'dt': 0.01, 'gamma': 0.2, 'n_samples_trn': 1024, 'ngf': 64, 'nstep': 2, 'final_tstep': 501, 'num_epochs': 10, 'n_primes': 100, 'learning_rate': 1e-05, 'device': 'cuda'}
no. of datasets: 25
cuda:0
model saved at: weights/model_unet_size_64_step_2_init_1_delta_250_tstep_250.pt
Start Training
Training Run: 0, prime: 2



Epoch [1/10], Training Loss: 26.55668243341



Epoch [2/10], Training Loss: 21.58521788465



Epoch [3/10], Training Loss: 21.32111103601



Epoch [4/10], Training Loss: 19.27267714665


...


Epoch [6/10], Training Loss: 8.47765772782



Epoch [7/10], Training Loss: 8.89775005929



Epoch [8/10], Training Loss: 10.20977569279



Epoch [9/10], Training Loss: 9.77959769409



Epoch [10/10], Training Loss: 8.79415833496
End Training


In [62]:
torch.save({"state":model.state_dict()}, "weights/model_unet_size_64_step_2_init_1_delta_250_tstep_250.pt")